# 목표: 완성된 모델을 Test set에 적용해 성능을 확인한다.

# Huggingface에서 다시 모델을 불러온다.

In [1]:
from transformers import pipeline

my_classifier = pipeline(
    "audio-classification",
    model="Hoonvolution/distilhubert-finetuned-hoons_music"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# 데이터셋도 다시 불러온다.

In [2]:
from datasets import load_dataset, Audio

dataset_id = 'Hoonvolution/hoons_music_data'
my_dataset = load_dataset(dataset_id, split='test', data_dir='/')
my_dataset = my_dataset.class_encode_column("artist")

Resolving data files:   0%|          | 0/2380 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/480 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/480 [00:00<?, ?it/s]

In [3]:
# id <-> label 매핑

id2label_fn = my_dataset.features['artist'].int2str

id2label = {
    i: id2label_fn(i)
    for i in range(len(my_dataset.features["artist"].names))
}
label2id = {v: k for k, v in id2label.items()}

### 테스트셋을 iterate하며 결과를 데이터프레임으로 저장

In [4]:
import pandas as pd
from tqdm import tqdm

result = []
for data in tqdm(my_dataset):
    audio = data['audio']
    predictions = sorted(my_classifier(audio), key=lambda x:x['score'], reverse=True)

    prediction, second_prediction, *_ = predictions
    true_label = id2label[data['artist']]
    result.append({
        'artist': true_label,
        'album': data['album'],
        'title': data['title'],
        'seg_id': data['seg_id'],
        'label_predicted': prediction['label'],
        'score': prediction['score'],
        'label_predicted_second': second_prediction['label'],
        'score_second': second_prediction['score'],
        })
df = pd.DataFrame(result)

100%|██████████| 480/480 [09:50<00:00,  1.23s/it]


In [5]:
df

,artist,album,title,seg_id,label_predicted,score,label_predicted_second,score_second
0,Epik High,"Pieces, Pt. 1",Décalcomanie,0,Epik High,0.668420,브로콜리 너마저,0.195390
1,Epik High,"Pieces, Pt. 1",Décalcomanie,1,Epik High,0.660127,브로콜리 너마저,0.275890
2,Epik High,"Pieces, Pt. 1",Décalcomanie,10,Epik High,0.951970,브로콜리 너마저,0.019147
3,Epik High,"Pieces, Pt. 1",Décalcomanie,11,Epik High,0.850517,Red Hot Chili Peppers,0.058045
4,Epik High,"Pieces, Pt. 1",Décalcomanie,12,Epik High,0.735625,Red Hot Chili Peppers,0.149867
...,...,...,...,...,...,...,...,...
475,브로콜리 너마저,졸업,커뮤니케이션의 이해,5,브로콜리 너마저,0.943689,Mika,0.023308
476,브로콜리 너마저,졸업,커뮤니케이션의 이해,6,브로콜리 너마저,0.642181,Fall Out Boy,0.142475
477,브로콜리 너마저,졸업,커뮤니케이션의 이해,7,브로콜리 너마저,0.901026,Mika,0.033270
478,브로콜리 너마저,졸업,커뮤니케이션의 이해,8,브로콜리 너마저,0.944105,Muse,0.018388


In [6]:
# 결과 요약

is_correct =  df['artist'] == df['label_predicted']
accuracy = 100 * is_correct.sum()/len(is_correct)
print(f"Accuracy: {accuracy:.2f} %")
is_correct.value_counts()

Accuracy: 77.08 %


,count
True,370
False,110


In [7]:
df.to_csv('result.csv', index=False)